# 3.2 Object-Oriented Design for Implementation

3.2.6 Exercises

## 1.

pass

## 2. Remove the `save_hyperparameters` statement in the B class. Can you still print `self.a` and `self.b`? Optional: if you have dived into the full implementation of the `HyperParameters` class, can you explain why?

In [2]:
from d2l import torch as d2l

# Call the fully implemented HyperParameters class saved in d2l
class B(d2l.HyperParameters):
    def __init__(self, a, b, c):
        # self.save_hyperparameters(ignore=['c'])
        print('self.a =', self.a, 'self.b =', self.b)
        print('There is no self.c =', not hasattr(self, 'c'))

b = B(a=1, b=2, c=3)

AttributeError: 'B' object has no attribute 'a'

Clearly we can not print `self.a` and `self.b`, then let's dive into the source code of the D2L library to see how it implements the HyperParameters class. 

The HyperParameters class of D2L Library is as follows:

In [9]:
import inspect # inspect module for getting information about live objects 
               # such as modules, classes, methods, functions, tracebacks, frame objects, and code objects.

class HyperParameters:
    """The base class of hyperparameters."""
    def save_hyperparameters(self, ignore=[]):
        """Defined in :numref:`sec_oo-design`"""
        raise NotImplemented

    def save_hyperparameters(self, ignore=[]):
        """Save function arguments into class attributes.
    
        Defined in :numref:`sec_utils`"""
        frame = inspect.currentframe().f_back # access the frame object of the caller's caller
        _, _, _, local_vars = inspect.getargvalues(frame) # access the locals dictionary of the given frame        
        ########## private addition ##########
        print("caller: ", inspect.currentframe()) # current frame object is the current function
        print("caller's caller: ", inspect.currentframe().f_back) # caller's caller frame object is the __init__ function
        print("locals: ", local_vars)
        ########## private addition ##########
        self.hparams = {k:v for k, v in local_vars.items()
                        if k not in set(ignore+['self']) and not k.startswith('_')} # get the variables that are not ignored and not private
        for k, v in self.hparams.items():
            setattr(self, k, v) # use setattr to add the variables to the class instance(not to the class)

Here, the `frame` denotes one frame in function calling stack(函数调用栈).If we want to access the frame related to a function, we must access it after calling that function and before returning from it(想要获得某个函数相关的帧，则必须在调用这个函数且这个函数尚未返回时获取).

The Frame object contains information about the caller function, including caller's caller, global variables, local variables and others.

In [10]:
class B(HyperParameters):
    def __init__(self, a, b, c):
        self.save_hyperparameters(ignore=['c'])
        print('self.a =', self.a, 'self.b =', self.b)
        print('There is no self.c =', not hasattr(self, 'c'))

b = B(a=1, b=2, c=3)

caller:  <frame at 0x7fdae23e8610, file '/tmp/ipykernel_8809/531645102.py', line 17, code save_hyperparameters>
caller's caller:  <frame at 0x7fdae23e8800, file '/tmp/ipykernel_8809/490287943.py', line 3, code __init__>
locals:  {'self': <__main__.B object at 0x7fdae24ea460>, 'a': 1, 'b': 2, 'c': 3}
self.a = 1 self.b = 2
There is no self.c = True
